Load sql extention and connect to database file

In [97]:
%load_ext sql
%sql sqlite:///software_project_management.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Drop all tables

In [98]:
%%sql
DROP TABLE IF EXISTS Project;
DROP TABLE IF EXISTS Employee;
DROP TABLE IF EXISTS Plan;
DROP TABLE IF EXISTS Activity;
DROP TABLE IF EXISTS EmployeeActivity;


 * sqlite:///software_project_management.db
Done.
Done.
Done.
Done.
Done.


[]

### Creates Employee table  
Constraints:
- employeeid is a primary key
- name and hourycost are not null
- hourly cost is a positive number

In [99]:
%%sql
CREATE TABLE IF NOT EXISTS Employee (
    employeeid INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    hourlycost FLOAT NOT NULL
    CHECK (hourlycost > 0)
);

 * sqlite:///software_project_management.db
Done.


[]

### Creates project table  
Constraints:
- projectid is a primary key
- no null values
- budget is a positive number
- leaderid is a foreign key referencing employeeid
- startdate is before enddate
- budget is more than 0


In [100]:
%%sql
CREATE TABLE IF NOT EXISTS Project (
    projectname TEXT PRIMARY KEY,
    leaderid INTEGER NOT NULL,
    budget FLOAT NOT NULL,
    startdate DATE NOT NULL,
    enddate DATE NOT NULL,
    FOREIGN KEY (leaderid) REFERENCES Employee(employeeid)
    CHECK (enddate > startdate AND budget > 0)
);

 * sqlite:///software_project_management.db
Done.


[]

### Creates Plan table  
Constraints:
- planid is a primary key
- no null values
- projectname is a foreign key referencing projectname (primary key in project table)
- startdate is before or the same as enddate (allowing one day plans)
- startdate and enddate are within the project start and end date (can be equal)

In [101]:
%%sql
CREATE TABLE IF NOT EXISTS Plan (
    planid INTEGER PRIMARY KEY,
    projectname TEXT NOT NULL,
    startdate DATE NOT NULL,
    enddate DATE NOT NULL,
    FOREIGN KEY (projectname) REFERENCES Project(projectname)
    CHECK (startdate <= enddate)
);

CREATE TRIGGER IF NOT EXISTS check_plan_dates
BEFORE INSERT ON Plan
FOR EACH ROW
BEGIN
    SELECT RAISE(ABORT, 'Plan dates are not within project dates')
    FROM Project
    WHERE Project.projectname = NEW.projectname
    AND (NEW.startdate < Project.startdate OR NEW.enddate > Project.enddate);
END;

 * sqlite:///software_project_management.db
Done.
Done.


[]

### Creates Activity table
Constraints:
- activityid is a primary key
- no null values
- planid is a foreign key referencing planid (primary key in plan table)
- startdate is before or the same as enddate (allowing one day activities)
- startdate and enddate are within the plan start and end date (can be equal)

In [102]:
%%sql
CREATE TABLE IF NOT EXISTS Activity (
    activityid INTEGER PRIMARY KEY,
    planid INTEGER NOT NULL,
    activitytype TEXT NOT NULL,
    startdate DATE NOT NULL,
    enddate DATE NOT NULL,
    FOREIGN KEY (planid) REFERENCES Plan(planid)
    CHECK (startdate <= enddate)
);

CREATE TRIGGER IF NOT EXISTS check_activity_dates
BEFORE INSERT ON Activity
FOR EACH ROW
BEGIN
    SELECT RAISE(ABORT, 'Activity dates are not within plan dates')
    FROM Plan
    WHERE Plan.planid = NEW.planid
    AND (NEW.startdate < Plan.startdate OR NEW.enddate > Plan.enddate);
END;

 * sqlite:///software_project_management.db
Done.
Done.


[]

### Creates EmployeeActivity table
Constraints:
- employeeid and activityid are a composite primary key
- employeeid is a foreign key referencing employeeid (primary key in employee table)
- activityid is a foreign key referencing activityid (primary key in activity table)
- no null values
- hours is a positive number

In [103]:
%%sql
CREATE TABLE IF NOT EXISTS EmployeeActivity (
    employeeid INTEGER NOT NULL,
    activityid INTEGER NOT NULL,
    hoursworked FLOAT NOT NULL,
    PRIMARY KEY (employeeid, activityid),
    FOREIGN KEY (employeeid) REFERENCES Employee(employeeid),
    FOREIGN KEY (activityid) REFERENCES Activity(activityid)
    CHECK (hoursworked > 0)
);

 * sqlite:///software_project_management.db
Done.


[]

### Add data to all tables
10 tuples in each table

In [104]:
%%sql
-- 10 records for Employee table
INSERT INTO Employee (name, hourlycost) VALUES ('James Johnson', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Benjamin Dover', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('John Smith', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Jane Doe', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Eel On Musk', 250.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Amo Guss', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Oliver Klozoff', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Bea O Problem', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Anita Bath', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Ima Weiner ', 220.50);

-- 10 records for Project table
INSERT INTO Project VALUES ('Project 1', 1, 10000.00, '2018-01-01', '2018-12-31');
INSERT INTO Project VALUES ('Fix twitter', 5, 200000.00, '2022-10-27', '2023-11-11');
INSERT INTO Project VALUES ('Project 3', 3, 10000.00, '2012-01-01', '2014-12-31');
INSERT INTO Project VALUES ('Project 4', 4, 10000.00, '2015-01-01', '2017-12-31');
INSERT INTO Project VALUES ('Project 5', 5, 10000.00, '2020-01-01', '2023-12-31');
INSERT INTO Project VALUES ('Project 6', 1, 50000.00, '2022-11-23', '2023-12-31');
INSERT INTO Project VALUES ('Project 7', 5, 10000.00, '2019-01-01', '2019-12-31');
INSERT INTO Project VALUES ('Project 8', 3, 80000.00, '2022-12-21', '2023-05-22');
INSERT INTO Project VALUES ('Project 9', 4, 10000.00, '2020-10-21', '2022-12-31');
INSERT INTO Project VALUES ('Project 10', 5, 50000.00, '2018-05-01', '2019-12-31');
INSERT INTO Project VALUES ('A', 5, 20000.00, '2019-03-10', '2019-03-30');

-- 10 records for Plan table
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 1', '2018-01-01', '2018-12-31');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Fix twitter', '2022-12-10', '2022-12-12');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 3', '2012-05-01', '2013-12-31');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 4', '2015-03-01', '2016-12-31');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 5', '2020-05-01', '2023-06-31');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 6', '2022-12-23', '2023-01-31');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 7', '2019-05-01', '2019-09-31');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 8', '2023-01-21', '2023-04-22');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 9', '2020-12-21', '2021-12-31');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('Project 10', '2018-12-01', '2019-02-31');
INSERT INTO Plan (projectname, startdate, enddate) VALUES ('A', '2019-03-15', '2019-03-20');

-- 10 records for Activity table
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (1, 'requirement analysis', '2018-01-01', '2018-01-31');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (2, 'architecture design', '2022-12-10', '2022-12-12');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (3, 'development', '2012-05-01', '2013-06-31');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (4, 'testing', '2015-03-01', '2016-04-31');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (5, 'deployment', '2020-05-02', '2020-06-31');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (6, 'requirement analysis', '2022-12-24', '2023-01-20');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (7, 'architecture design', '2019-05-02', '2019-06-30');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (8, 'development', '2023-01-22', '2023-02-20');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (9, 'testing', '2020-12-22', '2021-01-31');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (10, 'deployment', '2018-12-02', '2018-12-31');
INSERT INTO Activity (planid, activitytype, startdate, enddate) VALUES (11, 'requirement analysis', '2019-03-15', '2019-03-20');

-- 10 records for EmployeeActivity table
INSERT INTO EmployeeActivity VALUES (1, 1, 10.5);
INSERT INTO EmployeeActivity VALUES (2, 2, 3);
INSERT INTO EmployeeActivity VALUES (3, 3, 2);
INSERT INTO EmployeeActivity VALUES (4, 4, 120);
INSERT INTO EmployeeActivity VALUES (5, 5, 30);
INSERT INTO EmployeeActivity VALUES (6, 6, 0.45);
INSERT INTO EmployeeActivity VALUES (7, 7, 70);
INSERT INTO EmployeeActivity VALUES (8, 8, 12);
INSERT INTO EmployeeActivity VALUES (9, 9, 50);
INSERT INTO EmployeeActivity VALUES (10, 10, 4);

 * sqlite:///software_project_management.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]